In [11]:
import json
import sys
import threading
import websocket
import simplejson
from time import sleep
from datetime import datetime
import logging
import os
import PJS_RESTv2 as pjs


ModuleNotFoundError: No module named 'PJS_RESTv2'

In [5]:
logging.basicConfig()

In [6]:
#test funcion datetime
year = str(datetime.today().year)
day = str(datetime.today().day)
m=0
m+=1
if(len(day)==1):
    day = "0"+day
month = str(datetime.today().month)
if(len(month)==1):
    month = "0"+month
print(year)
print(month)
print(day)
print (m)

2018
11
10
1


In [8]:
listaMensajes=[]
subsMDEvents = []
subsOREvents = []
mensajes=0
MSG_MDSuscription = '{{"type":"smd","level":1,"entries":[{e}],"products":[{s}]}}'
MSG_OSSuscription = '{{"type":"os","account":{{"id":"{a}"}}}}'
#MSG_Symbol = '{{"symbol":"{s}","marketId":"ROFX"}}'
MSG_Double_Quotes = '"{s}"'
#global mensajes

In [9]:
def on_message(ws, message):
   # now  = datetime.now()
        
    try:
        print ("en el try")
        # Valido Mensaje entrante
        msg = simplejson.loads(message)
        msgType = msg['type'].upper()


        if msgType == 'MD':
            # Aca es Market Data, esta escuchando novedades
            
            #subsMDEvents.append(msg)
            print("----------------------------------------------")
            #print("Mensajes:---->", len(subsMDEvents))
            global mensajes
            mensajes+=1
            print("Mensajes:---->", mensajes)
            
            
            print ("En mensaje MD")
            #sys.stdout.flush()
            now  = datetime.now()
            print(str(now), msg)
           
                      
            
            #print("before processinf msg:...")
            symbol=msg['instrumentId']['symbol']
            
            #print (symbol)
            
            #Aca hay un problema si no hay bid u offer pq solo viene ['marketData']
            bidMsg=msg['marketData']['BI']
            offerMsg=msg['marketData']['OF']
            #print("BID msg :", bidMsg)
            #print("OFFER msg :",offerMsg)
            if bidMsg==[]:
                print("------------------------------------------>No BID detected")
                bid=0
                bidSize=0
            else:
                bid=msg['marketData']['BI'][0]['price']
                bidSize=msg['marketData']['BI'][0]['size']
            if offerMsg==[]:
                print("------------------------------------------>No OFFER detected")
                offer=0
                offerSize=0
            else:
                offer=msg['marketData']['OF'][0]['price']
                offerSize=msg['marketData']['OF'][0]['size']
                       
            
            msgArray=[]          
            msgArray.append(symbol)
           
            msgArray.append(bid)
            msgArray.append(bidSize)
            msgArray.append(offer)
            msgArray.append(offerSize)
            listaMensajes.append(msgArray)
           
            
            print(symbol,"----->", bid,"/", offer," SIZE--> ",bidSize,"/",offerSize)
            print("------------------------------------------------------")
            
                       
            
            if (bidSize ==7):
                print("BID Size of ", symbol,"----------------->",bidSize)
            
                #mensajes=0
                
            
            #print("---> Instrument Id",msg['instrumentId']['symbol'])
            #print("---> Market Data", msg['marketData'])
            #print("---> Market  Data BID ",msg['marketData']['BI'][0]['price'])
            #print("---> Market Data Offer",msg['marketData']['OF'])
            #  TODO, aca va todo lo que se quiere hacer con los msg recibidos
            # 
        
            # data = msg;

            #si queremos escribir a un archivo usamos algo asi:
            #filename.write(str(now)+"|"+str(data['marketData']['LA']['price'])+"|"+str(data['marketData']['LA']['size'])+"|"+str(data['marketData']['BI'][0]['price'])+"|"+str(data['marketData']['BI'][0]['size'])+"|"+str(data['marketData']['OF'][0]['price'])+"|"+str(data['marketData']['OF'][0]['size'])+"\n");
            #filename.close()


        elif msgType == 'OR':
            print ("En Mensaje OR")
            print(msg)
        else:
            print ("Tipo de Mensaje Recibido No soportado: " + msg)
    except:
        print ("Error al procesar mensaje recibido: " + message)

def on_error(ws, error):
    print ("Salio por error")
    print (error)
    ws.close()
    
def on_close(ws):
    print ("### connection closed ###")
    
def on_open(ws):    
    
    print ("On Conection Open...")
    

In [10]:
def initWS():
    # PARAMETROS para usuario y urls
    usr="balanz"
    pswd="balanz"
    account="555"
    endpointDemo = "http://demo-api.primary.com.ar/"
    wsEndpointDemo="ws://demo-api.primary.com.ar/"
    historyOHLC_endpoint = "http://h-api.primary.com.ar/MHD/TradesOHLC/{s}/{fi}/{ff}/{hi}/{hf}"
    #global mensajes
    #mensajes=0

    
    #parametros contratos a suscribir
    type_="smd"
    level_="1"
    symbol_="RFX20Dic18"
    
    symbol2="DoDic18"
    marketId_="ROFX"
    #aaa = "{\"type\":\"",type_,"\",\"level\":",level_,", \"entries\":[\"LA\", \"BI\", \"OF\"],\"products\":[{\"symbol\":\"",symbol_,"\",\"marketId\":\"",marketId_,"\"}]}"

    # Inicializamos con usuario/password/cuenta/
    #esta es la funcion: def init(userParam, passwordParam, accountParam, entornoParam, verifyHTTPsParam=False):
    #pmy.init("usuario", "pass", "10", pmy.Entorno.demo)
    #pjs.init("balanz", "balanz", "555",1) 

    
    pjs.setURLs(endpointDemo,wsEndpointDemo,historyOHLC_endpoint)
    pjs.init(usr,pswd,account,1)
    isLogin = pjs.login()

    if isLogin:
        
        # control
        print("Conexion establecida...")
        print('Token:'+' '+pjs.token)
        # end control
        
        headers = {'X-Auth-Token:{token}'.format(token=pjs.token)}

        ws = websocket.WebSocketApp(pjs.activeWSEndpoint,
                                         on_message=on_message,
                                         on_error=on_error,
                                         on_close=on_close,
                                         on_open=on_open,
                                         header=headers)

        wst = threading.Thread(target=ws.run_forever, kwargs={"ping_interval" :5})
        #wst = threading.Thread(target=ws.run_forever, kwargs={"ping_interval": 2}) #y nada
        # self.wst = threading.Thread(target=self.ws.run_forever())
        
        wst.start()

        # Esperamos a que la conexion ws se establezca
        conn_timeout = 5
        #conn_timeout = 50 #y nada
        sleep(1)


        while not ws.sock.connected and conn_timeout:
            sleep(1)
            conn_timeout -= 1
        else:
            
            # Aca se definen los prod a suscribir
            # tienen la forma:
            # aaa= "{\"type\":\"smd\",\"level\":1, \"entries\":[\"BI\", \"OF\"],\"products\":[{\"symbol\":\"DODic18\",\"marketId\":\"ROFX\"}]}"
            # pueden ser 1 a 1 creando diferentes mensajes
            # o en un solo mensaje con un array[] dentro del mensaje
            # pruebo con 2 mensajes aaa y bbb
            
            aaa = "{\"type\":\""+type_+"\",\"level\":"+level_+", \"entries\":[\"BI\", \"OF\"],\"products\":[{\"symbol\":\""+symbol_+"\",\"marketId\":\""+marketId_+"\"}]}"
            bbb=  "{\"type\":\""+type_+"\",\"level\":"+level_+", \"entries\":[\"BI\", \"OF\"],\"products\":[{\"symbol\":\""+symbol2+"\",\"marketId\":\""+marketId_+"\"}]}"
            
            
            #TEST: un mensaje de suscripcion por producto, se podrian meter en loop
            # se mandan los mensajes al websocket
            ws.send(aaa)
            ws.send(bbb)
                                    
            #mensajes enviados
            print("Sent")
            print(aaa)
            print(bbb)
            
            print("Receiving...")
            #result =  ws.recv()
            #print("Received '%s'" % result)
            sleep(1) #y nada

    else:
        print( "Error al autenticarse...")


if __name__ == "__main__":
    initWS()
    #ws = {"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"DOJul18","marketId":"ROFX"}]};
    #on_newmsg_ale()

    #on_message(ws, message):


    #if(ws.sock.connected):

NameError: name 'pjs' is not defined